## 十二. 分类(Classification)


以下是一个意图分类器：

In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

True

In [2]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json

INTENTS = [
    "play_internet_radio", 
    "play_song_by_artist",
    "get_weather",
    "current_time",
    "set_timer",
    "remind_me",
]

def build_intent_classifier(intents=INTENTS):
    template = (
        "Act as the intent classification component of a home assistant, similar to Amazon Alexa "
        "(except your name is 'Becca', not 'Alexa').\n"
        f"Common intents include: {', '.join(intents)}, ...\n"
        'You receive input in json format: `{{"input":...}}`\n'
        'You respond in json format: `{{"intent":..., "arguments":{{ ... }}, }}}}`\n\n'
        '{{"input":{spoken_request}}}'
    )

    llm = OpenAI(temperature=0.1)
    prompt = PromptTemplate(
        input_variables=["spoken_request"],
        template=template,
    )
    return LLMChain(llm=llm, prompt=prompt)

def evaluate(chain, text):
    response = chain.run(text)
    return response
#    return json.loads(response.strip())


In [3]:
chain = build_intent_classifier()

In [7]:
text = "becca play kexp"
response = evaluate(chain, text)
print(response)
#assert response == {'intent':'play_internet_radio', 'arguments':{'station':'KEXP'}}



{"intent":"play_internet_radio", "arguments":{"station":"KEXP"}}


In [8]:
text = "becca, how would I drive from my home to SeaTac airport?"
response = evaluate(chain, text)
print(response)



{"intent":"get_directions", "arguments":{"start_location":"home", "end_location":"SeaTac airport"}}


In [5]:
text = "Jerry, i'm hungry, let's go to lunch together"
response = evaluate(chain, text)
print(response)



{"intent": "remind_me", "arguments": {"time": "lunchtime", "message": "Let's go to lunch together"}}
